RNN PLAY GENERATOR

In [15]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
#Importovali smo Shakespere 
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
#Citamo fajl i vrsimo neku decodovanje enkodovanje nije bas jasno
text = open(path_to_file, 'rb').read().decode(encoding = 'utf8')
#Samo ispisujemo duzinu teksta
print ('Lenght of text: {} characters'.format(len(text)))

Lenght of text: 1115394 characters


In [4]:
#Samo da vidimo deo teksta
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
#Prvo moramo nas tekst da procesujemo, u nasem slucaju izvlacicemo sve jedinstvene karaktere
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
    return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [8]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
#Sad samo da napravimo funkciu koja radi obrnuto
#Iz intigera u rec
def int_to_text(int):
    try:
        int = int.numpy()
    except:
        pass
    return ''.join(idx2char[int])

print(int_to_text(text_as_int[:13]))

First Citizen


In [10]:
seq_lenght = 100
examples_per_epoch = len(text)//(seq_lenght + 1)

#Trening i test setovi
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [11]:
#Delimo nas trening skup u batch-eve odredjene duzine
sequences = char_dataset.batch(seq_lenght + 1, drop_remainder = True)

In [12]:
#te betch-eve sada moramo da podelimo na input i output vrednosti
def spit_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    
    return input_text, target_text

dataset = sequences.map(spit_input_target) #Na svakom clanu sequence ce se uraditi ova metoda

In [13]:
for x, y in dataset.take(2):
    print("\n\nEXAMPLE\n")
    print("INPUT")
    print(int_to_text(x))
    print("\nOUTPUT")
    print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [14]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

In [19]:
#Sada Pravimo model
def build_model(vocab_size, embeding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embeding_dim, batch_input_shape = [batch_size, None]), #Ovde stoji None jer neznamo kolike duzine ce biti nas batch sequence
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful = True, recurrent_initializer = 'glorot_uniform'), #Return intermidiate step value - return sequence
        tf.keras.layers.Dense(vocab_size) #Poslednji je dense iz razloga sto nas model ce moci da pretpostavi koji od 60000 karaktera moze biti sledeci u nase mtekstu
    ])
    
    return model

model = build_model(VOCAB_SIZE, EMBEDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [20]:
#Pravljenje sopstvene loss funkcije
for input_example_batch, target_example_batch in data.take(1):
    example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)
